In [ ]:
import sqlite3
import pandas as pd

def print_table(query):
    conn = sqlite3.connect('data/ratingRager.db')
    cursor = conn.cursor()
    cursor.execute(query)

    column_names = [description[0] for description in cursor.description]

    rows = cursor.fetchall()

    column_widths = [max(len(str(row[i])) for row in rows + [column_names]) for i in range(len(column_names))]

    header = "  ".join(f"{name:<{column_widths[i]}}" for i, name in enumerate(column_names))
    print('\n',header)
    print("-" * len(header))

    for row in rows:
        row_str = "  ".join(f"{str(item):<{column_widths[i]}}" for i, item in enumerate(row))
        print(row_str)
    print('\n')

    return pd.read_sql_query(query, conn)

In [ ]:
QUERY = """
SELECT r.id, r.name, r.rating, r.rating_volume
FROM restaurants as r
"""
df = print_table(QUERY)

In [ ]:
# Update score table

conn = sqlite3.connect('data/ratingRager.db')
cursor = conn.cursor()

for index, row in df.iterrows():
    cursor.execute(
        "UPDATE scores SET rating_score = ? WHERE restaurant_id = ?;",
        (row['overall_rating'], row['id'])
    )

conn.commit()
conn.close()

In [ ]:
median_rating_volume = df['rating_volume'].median()

average_rating_volume = df['rating_volume'].mean()

print("median: ", median_rating_volume)

print("mean: ", average_rating_volume)


median:  258.0
mean:  493.2795535261289


In [ ]:
df["overall_rating"] = (
    df["rating"]
    * df["rating_volume"].clip(upper=average_rating_volume)
    / average_rating_volume
    / 5
)

print(df["rating"][0], df["rating_volume"][0], average_rating_volume)
print(df["overall_rating"])

In [ ]:
print(df["overall_rating"].max())

1.0


In [ ]:
# Update score table

conn = sqlite3.connect('data/ratingRager.db')
cursor = conn.cursor()

for index, row in df.iterrows():
    cursor.execute(
        "UPDATE scores SET rating_score = ? WHERE restaurant_id = ?;",
        (row['overall_rating'], row['id'])
    )

conn.commit()
conn.close()